# Imporation des modules

In [24]:
###### modules pour le chargement des données depuis le XML ######
import glob
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

###### modules pour la classification ######

# modèles
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

# création de nos transformers
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer # créer nos propres transformer

# recherche des meilleurs hyperparamètres
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# résultats
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# sauvegarde des modèles
from joblib import dump, load

# spécifique pour l'analyse de sentiments
#%pip install transformers[sentencepiece]
#%pip install torch
from transformers import pipeline

###### modules pour la visualisation ######
import matplotlib.pyplot as plt
import pandas as pd

###### miscellaneous ######
from typing import List # typage des fonctions
import numpy as np
import re
from collections import namedtuple
import glob as glb

# Importation des fichiers

## Corpus politiques

In [14]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, _ = get_X_Y_from_root(root_test) # y_test est vide : pas accès aux résultats

In [15]:
# on récupère le y_test qui est dans un autre fichier

y_test = []
folder =  "./corpus/deft09_parlement_ref/"
files_ref = glb.glob(folder+"deft09_parlement_ref_fr.txt")[0]

with open(files_ref,'r') as file:
    line = file.readline()
    while line:
        line = line.split("\t")
        y_test.append(line[1])
        line = file.readline()

In [16]:
# échantillon pour tester plus rapidement

X_train_sample = X_train[:100]
y_train_sample = y_train[:100]
X_test_sample = X_test[:100]

# Analyse de sentiments

Pour l'instant, j'ai testé avec : 

- SVM : meilleur sans
- RandomForestClassifier : meilleur sans
- régression logistique : exactement la même chose

In [17]:
# Chargez le modèle de transformer pré-entraîné
sentiment_model = pipeline("sentiment-analysis", 
                           model="nlptown/bert-base-multilingual-uncased-sentiment") # uncased parce que ne prend pas en compte la casse

In [18]:
def sentiment_pred(posts):
    global sentiment_model
    return sentiment_model(posts, truncation=True)

In [25]:
# Ajoutez les prédictions de sentiments à votre pipeline de classification de discours politiques

pipeline_avec_sent = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
        ('sentiments', Pipeline([
          ('stats', FunctionTransformer(sentiment_pred)),
          ('vect', DictVectorizer())
        ]))
    ])),
    ('reglog', LogisticRegression())
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
pipeline_avec_sent.fit(X_train_sample, y_train_sample)

cv_scores_avec_sentiment = cross_val_score(pipeline_avec_sent, X_train_sample, y_train_sample)

In [26]:
print(np.mean(cv_scores_avec_sentiment))
print(cv_scores_avec_sentiment)

0.3

In [32]:
pipeline_sans_sent = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
    ])),
    ('reglog', LogisticRegression())
])

pipeline_sans_sent.fit(X_train_sample, y_train_sample)
cv_scores_sans_sent = cross_val_score(pipeline_sans_sent, X_train_sample, y_train_sample)

In [33]:
print(np.mean(cv_scores_sans_sent))
print(cv_scores_sans_sent)

array([0.4 , 0.35, 0.25, 0.3 , 0.2 ])

# Classification en thème

In [83]:
theme_classifier = pipeline("zero-shot-classification",
                      model="BaptisteDoyen/camembert-base-xnli")

candidate_labels = ["sport", 
                    "immigration", 
                    "économie", 
                    "sécurité",
                    "justice",
                    "science", 
                    "féminisme", 
                    "culture", 
                    "écologie", 
                    "climat",
                    "fiscalité",
                    "animal",
                    "social",
                    "agriculture",
                    "santé",
                    "travail",
                    "racisme"]
#hypothesis_template = "Ce texte parle de {}." # on peut peut-être utiliser ça pour renvoyer un dico

def theme_predictor(texts):
  """
  Prédire les thèmes dominants des textes donnés en utilisant un modèle de type "zero-shot" : BaptisteDoyen/camembert-base-xnli
  
  Parameters:
  texts (list): une liste de chaînes de caractères contenant les textes à classifier.
  
  Returns:
  list: une liste de dictionnaires, où chaque dictionnaire contient une entrée (thème, score) pour chaque thème prédit avec son score associé.
  """
  global theme_classifier
  global candidate_labels
  
  results = []
  for text in texts:
    classification = theme_classifier(text, candidate_labels)
    labels = classification["labels"]
    scores = classification["scores"]
    results.append({label:score for label, score in zip(labels, scores)})
      
  return results

In [85]:
# Ajoutez les prédictions de thèmes à votre pipeline de classification de discours politiques

pipeline_avec_theme = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
        ('sentiments', Pipeline([
          ('stats', FunctionTransformer(theme_predictor)),
          ('vect', DictVectorizer())
        ]))
    ])),
    ('linearsvc', LinearSVC())
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
#pipeline_avec_theme.fit(X_train_sample, y_train_sample)

cv_avec_theme = cross_val_score(pipeline_avec_theme, X_train_sample, y_train_sample, verbose=10)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] START .....................................................................


In [ ]:
print(np.mean(cv_avec_theme))
print(cv_avec_theme)

In [ ]:
pipeline_sans_theme = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
    ])),
    ('reglog', LogisticRegression())
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
#pipeline_sans_theme.fit(X_train_sample, y_train_sample)

cv_sans_theme = cross_val_score(pipeline_sans_theme, X_train_sample, y_train_sample, verbose=10)

In [ ]:
print(np.mean(cv_sans_theme))
print(cv_sans_theme)